<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Выбор перспективного региона для разработки нефтяных месторождений

**Цель проекта:** построить модель машинного обучения, которая поможет определить наиболее перспективный (с точки зрения прибыли) регион для разработки новых нефтяных месторождений. 

**Исходные данные:** пробы нефти в трёх регионах. В каждом регионе 10 000 месторождений, где измерили качество нефти и объём её запасов.

**Описание данных:**
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

**Условия задачи:**
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей,  поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
* При выполнении проекта рекомендуется использовать технику *Bootstrap*.

## Загрузка и подготовка данных

Загрузим необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

Загрузим исходные данные для трех регионов

In [2]:
source_data_1 = pd.read_csv("/datasets/geo_data_0.csv")
source_data_2 = pd.read_csv("/datasets/geo_data_1.csv")
source_data_3 = pd.read_csv("/datasets/geo_data_2.csv")

Выведем основную информацию об исходных датасетах

In [3]:
source_data_list = [source_data_1, source_data_2, source_data_3]
region_number = 0
for source_data in source_data_list:
    region_number += 1
    print("\nОсновная информация исходного датасета для региона {}:\n".format(region_number))
    print(source_data.info(), end="\n\n")
    print(source_data.head(), end="\n\n")
    print(source_data.describe(), end="\n\n")


Основная информация исходного датасета для региона 1:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848

Аномалий, включая пропуски, в исходных данных не выявлено, продолжим исследование. Подсчитаем количество полных дубликатов.

In [4]:
region = 0
for source_data in source_data_list:
    region += 1
    print("Количество полных дубликатов для региона {} равняется: {}\n".format(region, source_data.duplicated().sum()))

Количество полных дубликатов для региона 1 равняется: 0

Количество полных дубликатов для региона 2 равняется: 0

Количество полных дубликатов для региона 3 равняется: 0



Полных дубликатов не выявлено. Подсчитаем количество уникальных имен в столбцах трех исходных датасетов. Результаты подсчета выведем в отдельной таблице.

In [5]:
unique_names = pd.DataFrame(index=["Region_1", "Region_2", "Region_3"], columns=source_data_1.columns)
for source_data, i in zip(source_data_list, range(3)):
    for column, j in zip(source_data.columns, range(5)):
        unique_names.iloc[i, j] = len(source_data[column].unique())
print("\nТаблица 1.1. Уникальные имена в столбцах исходных датасетов (Регионы 1-3)")
unique_names


Таблица 1.1. Уникальные имена в столбцах исходных датасетов (Регионы 1-3)


,id,f0,f1,f2,product
Region_1,99990,100000,100000,100000,100000
Region_2,99996,100000,100000,100000,12
Region_3,99996,100000,100000,100000,100000


Как видно из таблицы, имеются повторы в столбцах id и product. Сперва рассмотрим на примере первого региона строки с повторяющимися id, для этого выведем их в отдельную таблицу.

In [6]:
mask = source_data_1["id"].duplicated()
id_duplicated = source_data_1.loc[mask, "id"]
id_duplicated_full = source_data_1.query("id in @id_duplicated").sort_values(by="id").reset_index()
print("\nТаблица 1.2. Скважины с совпадающими id для региона 1")
id_duplicated_full


Таблица 1.2. Скважины с совпадающими id для региона 1


,index,id,f0,f1,f2,product
0,66136,74z30,1.084962,-0.312358,6.990771,127.643327
1,64022,74z30,0.741456,0.459229,5.153109,140.771492
2,51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
3,3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
4,69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
5,42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
6,931,HZww2,0.755284,0.368511,1.863211,30.681774
7,7530,HZww2,1.061194,-0.373969,10.430210,158.828695
8,63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
9,1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502


Как видно из представленной таблицы, мы имеем 10 пар скважин с одинаковыми id. Для одних и тех же id разные запасы скважины. Причем для некоторых id данные запасы отличаются в несколько раз, например для id = HZww2 разница составляет более 5 раз. Вероятно, для ряда скважин пробы брались два раза и оценка запасов также осуществлялась два раза. Оценка запасов скважины, вероятно, является весьма приблизительной. С учетом того, что 10 совпадающих id составляют 0.01% от всех данных оставим датасет без изменения, а также другие датасеты (регионы 2 и 3), где повторов еще меньше. При этом желательно взять консультацию у составителей исходной таблицы по поводу повторов в ней и у технических специалистов по поводу того, как оцениваются запасы скважины (от чего зависит данная оценка и насколько она точная).

Теперь рассмотрим повторы в столбце product для 2 региона. Для этого выведем количество повторов для каждого из 12 уникальных значений.

In [7]:
product_duplicated = source_data_2["product"].value_counts().reset_index()
product_duplicated.sort_values(by="index", inplace=True)
product_duplicated.reset_index(inplace=True, drop=True)
product_duplicated.columns = ["unique_product_quantity", "number_of_duplicates"]
print("\nТаблица 1.3. Список уникальных значений запасов нефти для скважин региона 2")
product_duplicated


Таблица 1.3. Список уникальных значений запасов нефти для скважин региона 2


,unique_product_quantity,number_of_duplicates
0,0.000000,8235
1,3.179103,8337
2,26.953261,8468
3,30.132364,8306
4,53.906522,8472
5,57.085625,8390
6,80.859783,8320
7,84.038886,8431
8,107.813044,8201
9,110.992147,8303


Обращает на себя внимание ритмичность приращений запасов нефти скважин в отсортированной таблице при последовательном перемещении по списку от самого низкого запаса до самого большого. Добавим в данную таблицу дополнительный столбец delta с приращениями запасов скважин в отсортированном списке. Для этого напишем программу delta. 

In [8]:
def delta(column):
    new_column = pd.Series(np.nan)
    i = 1
    while i < len(column):
        new_column[i] = column.iloc[i] - column.iloc[i-1]
        i += 1
    return new_column

In [9]:
product_duplicated["delta"] = delta(product_duplicated["unique_product_quantity"])
print("\nТаблица 1.4. Отредактированный список уникальных значений запасов нефти для скважин региона 2")
product_duplicated


Таблица 1.4. Отредактированный список уникальных значений запасов нефти для скважин региона 2


,unique_product_quantity,number_of_duplicates,delta
0,0.000000,8235,NaN
1,3.179103,8337,3.179103
2,26.953261,8468,23.774158
3,30.132364,8306,3.179103
4,53.906522,8472,23.774158
5,57.085625,8390,3.179103
6,80.859783,8320,23.774158
7,84.038886,8431,3.179103
8,107.813044,8201,23.774158
9,110.992147,8303,3.179103


Из таблицы хорошо видно, что мы имеем ритмичные приращения на 3,18, а затем на 23,77 тысяч бараллей. В данном исследовании оставим столбец product для региона 2 без изменений. Однако отметим о желательности консультации с составителями таблицы и техническими специалистами по поводу природы выявленных аномалий в исходном датасете.

С целью подготовки данных к машинному обучению удалим столбец "id", так как уникальный id не влияет на целевой признак. В качестве же уникального индентификатора скважины будет выступать индекс датафрейма.

In [10]:
data_1 = source_data_1.drop("id", axis=1)
data_2 = source_data_2.drop("id", axis=1)
data_3 = source_data_3.drop("id", axis=1)

### Выводы

1. Загружены исходные данные для трех регионов. Исходные датасеты для всех регионов имеют одинаковый размер 100000 х 5.
2. Пропуски в исходных данных не выявлены.
3. Выявлены повторы в столбцах id для регионов 1, 2 и 3. От общего количества строк данные повторы не превышают 0,01%. Вследствие этого данные повторы оставлены без изменений.
4. В столбце product только 12 уникальных значений. Причина этого не ясна. Необходимы консультации с составителями таблицы и техническими специалистами.
5. С целью подготовки к машинному обучению в исходных данных удален столбец "id", так как уникальный индентификатор не влияет на целевой признак.

## Обучение и проверка модели

Создадим список list_of_vars, содержащий названия необходимых переменных для регионов 1-3.

In [11]:
list_of_vars_draft = ["features_", "target_", "features_train_", "features_valid_", "target_train_",
                      "target_valid_", "predicted_valid_"]
list_of_vars = []
for regions in range(1, 4):
    current_list = [x + str(regions) for x in list_of_vars_draft]
    list_of_vars.append(current_list)
list_of_vars

[['features_1',
  'target_1',
  'features_train_1',
  'features_valid_1',
  'target_train_1',
  'target_valid_1',
  'predicted_valid_1'],
 ['features_2',
  'target_2',
  'features_train_2',
  'features_valid_2',
  'target_train_2',
  'target_valid_2',
  'predicted_valid_2'],
 ['features_3',
  'target_3',
  'features_train_3',
  'features_valid_3',
  'target_train_3',
  'target_valid_3',
  'predicted_valid_3']]

Выполним обучение и проверку моделей для регионов 1-3. Для оптимизации кода и избежании повторов в нем воспользуемся циклом. Переменные с названиями из списка list_of_vars сохраним в словаре dict_of_vars.

In [12]:
list_of_data = [data_1, data_2, data_3]
R2 = []
RMSE = []
predicted_mean = []
actual_mean = []
dict_of_vars = {}
for region, data in zip(range(0, 3), list_of_data):
    dict_of_vars[list_of_vars[region][0]] = data.drop("product", axis=1)
    dict_of_vars[list_of_vars[region][1]] = data["product"]
    dict_of_vars[list_of_vars[region][2]], dict_of_vars[list_of_vars[region][3]], dict_of_vars[list_of_vars[region][4]], dict_of_vars[list_of_vars[region][5]] = train_test_split(
        dict_of_vars[list_of_vars[region][0]], dict_of_vars[list_of_vars[region][1]], test_size=0.25, random_state=12345)
    dict_of_vars[list_of_vars[region][5]].reset_index(inplace=True, drop=True)
    model = LinearRegression() 
    model.fit(dict_of_vars[list_of_vars[region][2]], dict_of_vars[list_of_vars[region][4]])
    predicted_valid = pd.Series(model.predict(dict_of_vars[list_of_vars[region][3]]))
    dict_of_vars[list_of_vars[region][6]] = predicted_valid
    R2.append(r2_score(dict_of_vars[list_of_vars[region][5]], predicted_valid))
    RMSE.append(mean_squared_error(dict_of_vars[list_of_vars[region][5]], predicted_valid)**0.5)
    predicted_mean.append(predicted_valid.mean())   
    actual_mean.append(dict_of_vars[list_of_vars[region][5]].mean())

Выгрузим переменные из словаря dict_of_vars.

In [13]:
for key, val in dict_of_vars.items():
    exec(key + "=val")

Создадим таблицу с основными результатами проверки модели.

In [14]:
results_dict = {"R2": R2, "RMSE": RMSE, "predicted_mean": predicted_mean, "actual_mean": actual_mean}
results = pd.DataFrame(results_dict, index=["Region_1", "Region_2", "Region_3"])
results = pd.DataFrame(results_dict, index=["Region_1", "Region_2", "Region_3"])
print("\nТаблица 2.1. Основные результаты проверки моделей для регионов 1-3")
results


Таблица 2.1. Основные результаты проверки моделей для регионов 1-3


,R2,RMSE,predicted_mean,actual_mean
Region_1,0.279943,37.579422,92.592568,92.078597
Region_2,0.999623,0.893099,68.728547,68.723136
Region_3,0.205248,40.029709,94.965046,94.884233


Как видно из таблицы 2.1, коэффициент детерминации составляет  0.28, 0.9996 и 0.21 для регионов 1, 2 и 3 соответственно. Для всех регионов коэффициент детерминации больше нуля. Таким образом, получившиеся модели превосходят модель, в которой целевой признак постоянен и рассчитывается как среднее значение всех целевых признаков. При этом обращает внимание на себя аномально высокое значение коэффициента детерминации для региона 2 равное 0.9996. Похоже, что значения целевого признака для региона 2 были определены не опытным путем, а с использованием линейной регрессии. 

### Выводы

1. Обучены модели для регионов 1, 2 и 3.
2. Коэффициент детерминации составляет  0.28, 0.9996 и 0.21 для регионов 1, 2 и 3 соответственно. Обращает внимание на себя аномально высокое значение коэффициента детерминации для региона 2 равное 0.9996. Похоже, что значения целевого признака для региона 2 были определены не опытным путем, а с использованием линейной регрессии на основание признаков f1, f2 и f3.

## Подготовка к расчёту прибыли

Зададим переменные для расчета прибыли.

In [15]:
BUDGET = 10000000
POINTS_SURVEY = 500
BOREHOLES = 200
PRICE_FOR_PRODUCT = 450

Рассчитаем цену за бурение одной скважины.

In [16]:
price_for_borehole_drilling = BUDGET / BOREHOLES
print("Цена за бурение одной скважины:", price_for_borehole_drilling / 1000, "млн. рубл.")

Цена за бурение одной скважины: 50.0 млн. рубл.


Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [17]:
necessary_oil_reserves = price_for_borehole_drilling / PRICE_FOR_PRODUCT
print("Достаточный объём сырья для безубыточной разработки новой скважины:", round(necessary_oil_reserves, 1), "тыс. бараллей")

Достаточный объём сырья для безубыточной разработки новой скважины: 111.1 тыс. бараллей


Рассчитаем средние запасы нефти по регионам, приходящиеся на одну скважину.

In [18]:
for data, region in zip(list_of_data, range(1, 4)):
    print("Средний запас нефти в регионе {} на одну скважину: {} тыс. бараллей".format(region, round(data["product"].mean(), 1)))

Средний запас нефти в регионе 1 на одну скважину: 92.5 тыс. бараллей
Средний запас нефти в регионе 2 на одну скважину: 68.8 тыс. бараллей
Средний запас нефти в регионе 3 на одну скважину: 95.0 тыс. бараллей


Согласно выполненным расчетам, средний запас нефти в регионах 1, 2 и 3 на одну скважину меньше требуемого, исходя из условия безубыточной разработки скважины. 

Напишем функцию для расчета прибыли по выбранным скважинам и предсказаниям модели. Программа принимает на вход количество отбираемых скважин и два списка значений целевого признака: один фактический и второй, согласно предсказанию, сделанному моделью. Возвращает функция значение прибыли (выручка от продажи сырья минус затраты на бурение скважин).

In [19]:
def profit_selected_boreholes(target, predictions, boreholes):
    predictions_selected = predictions.sort_values(ascending=False)[:boreholes]
    target_selected = target[predictions_selected.index]
    profit = target_selected.sum() * PRICE_FOR_PRODUCT - price_for_borehole_drilling * BOREHOLES 
    print(len(target_selected))
    return profit

### Выводы

1. Достаточный объём сырья для безубыточной разработки новой скважины равняется 111 тысячам бараллей. При этом средний запас нефти в регионах 1, 2 и 3 равняется 93, 69 и 95 тысячам бараллей. Таким образом, если отбирать скважины для бурения случайным образом, то фирма понесет убытки.
2. Написана функция для расчета прибыли по выбранным скважинам и предсказаниям модели.

## Расчёт прибыли и рисков 

Расчитаем распределение прибыли для регионов 1, 2 и 3, используя технику Bootstrap. Для оптимизации кода и избежании повторов в нем воспользуемся циклом. Результаты расчетов прибыли сохраним в списке profit_list.

In [20]:
profit_1 = []
profit_2 = []
profit_3 = []
profit_list = [profit_1, profit_2, profit_3]
state = np.random.RandomState(12345)
bootstrap_samples = 1000
for region in range(3):
    exec("target_valid = target_valid_" + str(region + 1))
    exec("predicted_valid = predicted_valid_" + str(region + 1))
    for i in range(bootstrap_samples):
        predictions_sample = predicted_valid.sample(n=POINTS_SURVEY, replace=True, random_state=state)
        exec("profit_" + str(region + 1) + ".append(profit_selected_boreholes(target_valid, predictions_sample, BOREHOLES))")

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


Поменяем тип данных, содержащихся в листе profit_list с list, на Series.

In [21]:
profit_list = list(map(lambda x: pd.Series(x), profit_list))

Рассчитаем для регионов 1-3 следующие параметры: среднюю прибыль, 95% доверительный интервал для прибыли и риск убытков.

In [22]:
for i, j in zip(["первого", "второго", "третьего"], profit_list):
    print("Средня прибыль для {} региона {:.1f} (млн. рубл.).".format(i, j.mean() / 1000))
    print("95% доверительный интервал для прибыли {} региона: от {:.1f} до {:.1f} (млн. рубл.).".format(
    i, j.quantile(0.025), j.quantile(0.975)))
    print("Риск убытков для {} региона: {:.1f} %.".format(i, j[j < 0].count() / 1000 * 100))
    print()

Средня прибыль для первого региона 396.2 (млн. рубл.).
95% доверительный интервал для прибыли первого региона: от -111215.5 до 909766.9 (млн. рубл.).
Риск убытков для первого региона: 6.9 %.

Средня прибыль для второго региона 461.2 (млн. рубл.).
95% доверительный интервал для прибыли второго региона: от 78050.8 до 862952.1 (млн. рубл.).
Риск убытков для второго региона: 0.7 %.

Средня прибыль для третьего региона 393.0 (млн. рубл.).
95% доверительный интервал для прибыли третьего региона: от -112227.6 до 934562.9 (млн. рубл.).
Риск убытков для третьего региона: 6.5 %.



### Выводы

1. Рассчитано распределение прибыли для регионов 1, 2 и 3 с использованием техники Bootstrap.
2. Средняя прибыль для регионов 1-3 составляет 396, 461 и 393 млн. рублей соответственно.
3. Верхняя граница 95% доверительного интервала для прибыли для регионов 1-3 составляет 909767, 862952 и 934563 млн. рублей соответственно. Нижняяя граница 95% доверительного интервала для прибыли для регионов 1-3 составляет -111216 , 78051 и -112228 млн. рублей соответственно.
4. Риск убытков для регионов 1-3 составляет 6.9, 0.7 и 6.5% соответственно.
5. С учетом полученных результатов расчетов регион 2 является самым предпочтительным для разработки месторождений. У данного региона минимальные риски убытков и самая высокая средняя прибыль. Однако исходные данные для данного региона выглядят аномальными: всего 12 уникальных значений в столбце product на 100 000 строк, коэффициент детерминации равный почти 1. С учетом этого необходимо выяснить причину указанных аномалий. После этого можно будет принять окончательное решение о выборе региона для разработки месторождений. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован